## Estado

El estado es una matriz de adyacencia, que indicara cuales fragmentos de pista estan conectados, por ejemplo si la matriz es de $2 \times 2 $, es decir tenemos solo dos piezas de carretera: 
$$
\begin{pmatrix}
1 & 0 \\
0 & 1 
\end{pmatrix}
$$
No nos interesa la diagonal de la matriz, porque las piezas no pueden estar conectadas hacia si mismos.

Nuestra matriz sera de $ 32 \times 32 $ porque tenemos 32 fragmentos  de piezas 

## Estado inicial

In [ ]:
# importamos las librererias necesarias y configuramos manim, libreria necesaria para animaciones
import numpy as np
from manim import *
import random
config.media_width = "100%"

# Declaramos las contantes, las cantidades de piezas de cada tipo
RECTAS = 12
CURVAS = 16
DISECCION_1 = 2
DISECCION_2 = 2

#Inicializamos estado inicial, una matriz identidad de 32 x 32
matriz_Adyescencia = np.eye(32)

# Accion

La accion es una tupla de enteros del 1 al 4, porque son 4 tipos de pista. 
1.  Pista Recta
2.  Pista Curva
3.  Pista Dissecion tipo 1 
4.  Pista Dissecion tipo 2

Estos numeros son seleccionados aleatoriamente

In [ ]:
# Funcion para hacer una accion
def Accion():
    return random.randint(1,4), random.randint(1,4)

# Funcion de Transicion
Esta funcion debera recibir un estado y una accion, esta funcion debera verificar si la accion es posible y si es posible regresara el estado dada la accion.
Para sabes si la accion posible deberan 
- La suma de la columna debera ser menor a dos 

In [168]:
def verificar_suma_columnas(matriz):
    # Verificar las primeras 28 columnas (índices 0 a 27)
    for j in range(28):
        suma_columna = sum(matriz[i][j] for i in range(32))
        if suma_columna > 2:
            print(f"Columna {j+1}: suma = {suma_columna} (debe ser ≤ 2)")
            return False
    
    # Verificar las columnas 29 a 32 (índices 28 a 31)
    for j in range(28, 32):
        suma_columna = sum(matriz[i][j] for i in range(32))
        if suma_columna > 3:
            print(f"Columna {j+1}: suma = {suma_columna} (debe ser ≤ 3)")
            return False
    return True


            
def rangos(tipo):
    if tipo == 1:
        return range(RECTAS)
    elif tipo == 2:
        return range(RECTAS, RECTAS + CURVAS)
    elif tipo == 3:
        return range(RECTAS + CURVAS, RECTAS + CURVAS + DISECCION_1)
    elif tipo == 4:
        return range(RECTAS + CURVAS + DISECCION_1,RECTAS + CURVAS + DISECCION_1 + DISECCION_1)
    else:
        return None
    
def Casilla_valida(matriz_np, tipo_1, tipo_2):

    # Definir el rango de búsqueda
    rango_1 = rangos(tipo_1)
    rango_2 = rangos(tipo_2)
    # Calcular sumas (excluyendo diagonal)
    sumas_columnas = np.sum(matriz_np, axis=0) - np.diag(matriz_np)
    sumas_filas = np.sum(matriz_np, axis=1) - np.diag(matriz_np)
    print(f"rango :{rango_1} rango 2: {rango_2}")
    # Buscar columnas válidas
    for i in rango_1:
        if sumas_columnas[i] == 0:
            for j in rango_2:
                if sumas_filas[j] == 0:
                    if i != j and (matriz_np[i][j] != 1 and matriz_np[j][i] != 1):
                        print(f"matriz[i][j] {matriz_np[i][j]} matriz[j][i] {matriz_np[j][i]}")
                        return i,j
    
    return None

In [169]:
def Funcion_transicion(matriz, accion):

    if accion == 1:
        return 0
    elif accion == 2:
        return 0
    elif accion == 3:
        return 0 
    elif accion == 4:
        return 0

# Pruebas

In [ ]:
matriz_Adyescencia = np.eye(32)
matriz_Adyescencia[1][0] = 1

print(matriz_Adyescencia)
accion = Accion()
print(f"a:{accion[0]}  b:{accion[1]}")
#i,j = Casilla_valida(matriz_Adyescencia, accion[0],accion[1])
i,j = Casilla_valida(matriz_Adyescencia,accion[0],accion[1])

print(f"i:{i} j:{j}")

[[1. 0. 0. ... 0. 0. 0.]
 [1. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
a:1  b:2
rango :range(0, 12) rango 2: range(12, 28)
matriz[i][j] 0.0 matriz[j][i] 0.0
i:1 j:12
